In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

val = df.copy()

# ensure numeric
val["years_since_review"] = pd.to_numeric(val["years_since_review"], errors="coerce")
val["NumberSubmitters_num"] = pd.to_numeric(val["NumberSubmitters_num"], errors="coerce")
val["conflicting"] = val["conflicting"].astype(bool)

# bins for staleness
bins = [0, 1, 2, 5, 10, 20, 50, 200]
labels = ["<1y", "1-2y", "2-5y", "5-10y", "10-20y", "20-50y", "50y+"]

val["ysr_bin"] = pd.cut(val["years_since_review"], bins=bins, labels=labels, include_lowest=True, right=False)

# simple groups
val["submitter_bin"] = pd.cut(
    val["NumberSubmitters_num"],
    bins=[-0.1, 1, 2, 5, 10, 10**9],
    labels=["1", "2", "3-5", "6-10", "11+"]
)

# summary table
sanity_tbl = pd.DataFrame({
    "group": [
        "conflicting = True",
        "conflicting = False",
        "single submitter (<=1)",
        "multi submitter (>=2)"
    ],
    "n": [
        int(val[val["conflicting"]].shape[0]),
        int(val[~val["conflicting"]].shape[0]),
        int(val[val["NumberSubmitters_num"] <= 1].shape[0]),
        int(val[val["NumberSubmitters_num"] >= 2].shape[0]),
    ],
    "mean_risk": [
        float(val.loc[val["conflicting"], "model_risk_proba"].mean()),
        float(val.loc[~val["conflicting"], "model_risk_proba"].mean()),
        float(val.loc[val["NumberSubmitters_num"] <= 1, "model_risk_proba"].mean()),
        float(val.loc[val["NumberSubmitters_num"] >= 2, "model_risk_proba"].mean()),
    ],
}).round(4)

sanity_path = OUT_TABLES / "validation_sanity_group_means.csv"
sanity_tbl.to_csv(sanity_path, index=False)

sanity_tbl, sanity_path


NameError: name 'df' is not defined

In [2]:
from pathlib import Path
import pandas as pd

def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start] + list(start.parents):
        if (p / "outputs").exists() and (p / "notebooks").exists():
            return p
    raise FileNotFoundError("Could not find repo root containing 'outputs' and 'notebooks'.")

ROOT = find_repo_root(Path.cwd())
SCORED_PATH = ROOT / "outputs" / "phase4" / "tables" / "phase4_scored.csv.gz"
print("ROOT:", ROOT)
print("SCORED_PATH:", SCORED_PATH)

assert SCORED_PATH.exists(), f"Missing scored file: {SCORED_PATH}"
df = pd.read_csv(SCORED_PATH, compression="gzip", low_memory=False)
print("Loaded df:", df.shape)


ROOT: /Users/sanghati/research/accountable-interpretation
SCORED_PATH: /Users/sanghati/research/accountable-interpretation/outputs/phase4/tables/phase4_scored.csv.gz
Loaded df: (300000, 52)
